# Brownian Motion
We will explore the properties of Brownian motion in 2 dimension.

Import useful modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.style as style
#style.use('dark_background')

In [15]:
# uncomment the following line if you want to use qt5 to visualize the results
# %matplotlib qt5

Let's start by considering a single particle undergoing Brownian motion. The parameters of the motion can be specified before the simulation. As in the previous assignments, we are using simplified units 

In [4]:
b = 0.1 # scaled coefficient of linear drag, b=b1/m
force = 0.005 # scaled magnitude of the random force F=f/m
tau = 2.0 # relaxation time of the random force

Now we can run some steps of a dynamics. We will use the following algorithm: 
1. first draw the time of the next random kick
2. if the kick time is after the end of the current timestep, just take a step of linear damped dynamic
3. otherwise:
   1. evolve the linear damp dynamics up to the time of the kick
   2. change the velocity according to the random force 
   3. draw the time of the next kick

To make the code more modular, we first create a function to evolve the system when there are no random kicks

In [5]:
def analytic_ld_dynamics(pos,vel,b,dt):
    """
    Analytic solution for the dynamics with linear damp.
    Given a particle with position pos and velocity vel in 2D
    and the scaled linear damp coefficient (b=b1/m)
    calculate the position and velocity after a timestep dt
    using the analytic expression (Eq 10.14 and 3.12-3.13 of the textbook)
    """
    edt=np.exp(-b*dt)
    return pos+vel*(1-edt)/b,vel*edt

As in the past, we will simulate the trajectory with a finite timestep. However, in this case, we are not using numerical integrators, as the linear damp dynamics has an analytic solution. The timestep is only used for visualization porposes. As initial configuration we can consider a particle in the origin with no velocity.

In [7]:
tkick=-np.log(np.random.random())*tau
t = 0
dt = 0.5
nsteps = 10000
pos=np.zeros(2)
vel=np.zeros(2)
x=[pos[0]] # containers to store the coordinates of the system
y=[pos[1]] # containers to store the coordinates of the system
for i in range(nsteps):
    tnext = t + dt
    while tkick <= tnext : # this check may happen more than once, if the new kick time is still within the same timestep
        dtkick = tkick - t 
        pos, vel = analytic_ld_dynamics(pos,vel,b,dtkick) # take a step up to the kick
        t += dtkick
        # apply the random force
        theta = np.random.random()*2.*np.pi # pick a random angle for the force
        vel += force*np.array([np.cos(theta),np.sin(theta)]) # update the velocity 
        # update the next kick time
        tkick = t - np.log(np.random.random())*tau # draw a new kick-time 
    dtleft = tnext - t
    pos, vel = analytic_ld_dynamics(pos,vel,b,dtleft) # evolve the system up to the end of the timestep
    t += dtleft
    x.append(pos[0])
    y.append(pos[1])

If we visualize it we can see that it looks quite similar to the random walk. 

In [9]:
plt.plot(x,y)
plt.axis('equal')
plt.show()

We can implement the same process as a Phyton class. A class (aka an object) has attributes (internal variables) and methods (functions that act on the instance of that class). The main definition of a class consist in the definition of its `__init__()` method

In [17]:
class brownian_motion():
    def __init__(self, N=400, F=0.005, b=0.1, tau=2., dt=0.5):
        """
        Initialize an instance of a system of N identical particles
        undergoing brownian motion with parameters F, b, and tau.
        The initial positions and velocities of the particles are set 
        to zero. For visualization purposes, the simulation is 
        split into timesteps of size dt
        """
        self.N = N
        self.F = F
        self.b = b
        self.tau = tau
        self.dt = dt
        self.pos = np.zeros((N,2))
        self.vel = np.zeros((N,2))
        self.tkick = -np.log(np.random.random(N))*tau

    def step(self,t):
        """
        For each particle in the system, perform a step
        of Brownian motion
        """
        tnext = t + self.dt
        for i in range(self.N): # loop over each particle
            tlocal = t # now we need to keep a local copy of the time, for each particle
            while self.tkick[i] <= tnext : # this check may happen more than once, if the new kick time is still within the same timestep
                dtkick = self.tkick[i] - tlocal 
                self.pos[i], self.vel[i] = analytic_ld_dynamics(self.pos[i],self.vel[i],self.b,dtkick) # take a step up to the kick
                tlocal += dtkick
                # apply the random force
                theta = np.random.random()*2.*np.pi # pick a random angle for the force
                self.vel[i] += self.F*np.array([np.cos(theta),np.sin(theta)]) # update the velocity 
                # update the next kick time
                self.tkick[i] = tlocal - np.log(np.random.random())*tau # draw a new kick-time 
            dtleft = tnext - tlocal
            self.pos[i], self.vel[i] = analytic_ld_dynamics(self.pos[i],self.vel[i],b,dtleft) # evolve the system up to the end of the timestep
        return tnext
    

With the class above, we can simulate a system of N identical particles under the effect of random forces.

In [14]:
t=0.
bm=brownian_motion(10,0.005,b,tau,dt)
x=[bm.pos[:,0].copy()] # containers to store the coordinates of the system
y=[bm.pos[:,1].copy()] # containers to store the coordinates of the system
for i in range(10000):
    t=bm.step(t) # using the step() method will change the attributes of the class and return the next timestep
    x.append(bm.pos[:,0].copy()) # containers to store the coordinates of the system
    y.append(bm.pos[:,1].copy()) # containers to store the coordinates of the system
plt.plot(x,y)
plt.axis('equal')
plt.show()

Assignment 2B: modify the above simulation to compute the mean squared displacement as a function of time, averaging over the $N$ identical realization of the motion.

Assigment 2C: in a similar way compute the mean squared velocity of the particle as a function of time. Plot the results and estimate $kT$ from the equipartition theorem.

Reduce $\tau$ by a factor of 2 and compare the mean squared displacement and the temperature with the previous simulation. Do the same with the kick strength (the `force` parameter)